# Prepare TS input for Arkane

In [ ]:
import os
import subprocess
from rmgpy.molecule.molecule import Molecule

from easy_rmg_model.species.info import (classify_jobs,
                                           find_latest_terminated_job,
                                           check_converge_and_geom_consist,
                                           generate_geom_info,
                                           find_rotors_from_xyz,
                                           filter_scans,
                                           check_scan_quality,
                                           generate_summary,)

from easy_rmg_model.template_writer.input import ArkaneSpecies

from arc.parser import parse_1d_scan_energies
from arc.plotter import plot_1d_rotor_scan

In [ ]:
def change_to_relative_path(spc, work_dir='.', get_back=True):
    """
    Change the path to relative path
    """
    # Get the current dir
    cur_dir = os.getcwd() if get_back else None

    # Point to the working directory
    os.chdir(work_dir)
    
    # Change job directories for composite, sp, freq
    for job_type in ['composite', 'sp', 'freq']:
        try:
            path = spc[job_type]
            spc[job_type] = os.path.relpath(path)
        except KeyError:
            pass
    
    # Change job directories for rotor scans
    try:
        rotors_dict = spc['rotors_dict']
    except KeyError:
        pass
    
    for ind, rotor in rotors_dict.items():
        if rotor['scan_path']:
            path = rotor['scan_path']
            rotor['scan_path'] = os.path.relpath(path)
    
    if get_back:
        os.chdir(cur_dir)

In [ ]:
# Write Arkane kinetics input according to Template
from arkane.encorr.corr import assign_frequency_scale_factor
from arkane.input import process_model_chemistry
from arkane.modelchem import LOT

from easy_rmg_model.template_writer import BaseTemplateWriter


class ArkaneKinetics(BaseTemplateWriter):

    default_settings = {
        'model_chemistry': 'cbs-qb3',
        'freq_scale_factor': None,
        'use_bond_corrections': False,
        'use_hindered_rotors': True,
        'reactants': {'': ''},
        'products': {'': ''},
        'TS': 'TS.py',
        'tunneling': 'Eckart',
        'template_file': None,
        'save_path': './input.py'
    }

    default_template = """#!/usr/bin/env python3
# encoding: utf-8

title = "{{ reaction_label | safe }}"

description = \
'''
{{ reaction_label | safe }}
'''
modelChemistry = "{{ model_chemistry }}"
frequencyScaleFactor = {{ freq_scale_factor }}
useHinderedRotors = {{ use_hindered_rotors }}
useBondCorrections = {{ use_bond_corrections }}
{% for smi, path in reactants %}
species("{{ smi | safe}}",
        "{{ path | safe }}",
        structure=SMILES("{{ smi | safe }}"),
        )
{%- endfor %}
{% for smi, path in products %}
species("{{ smi | safe }}",
        "{{ path | safe }}",
        structure=SMILES("{{ smi | safe }}"),
        )
{%- endfor %}

transitionState('TS', '{{ TS | safe }}')

reaction(
    label='{{ reaction_label | safe }}',
    reactants={{ reactant_list | safe }},
    products={{ product_list | safe }},
    transitionState = 'TS',
    tunneling='{{ tunneling }}',
)

kinetics(
    label='{{ reaction_label | safe }}',
    Tmin = (300,'K'), Tmax = (2000,'K'), Tcount = 200,
)

"""

    @property
    def model_chemistry(self):
        return self._model_chemistry

    @model_chemistry.setter
    def model_chemistry(self, value):
        if isinstance(value, str):
            self._model_chemistry = process_model_chemistry(value)
        elif isinstance(value, LOT):
            self._model_chemsitry = value

    @property
    def freq_scale_factor(self):
        if not self._freq_scale_factor:
            self._freq_scale_factor = assign_frequency_scale_factor(
                self._model_chemistry)
        return self._freq_scale_factor

    @freq_scale_factor.setter
    def freq_scale_factor(self, value):
        if value == None or \
           (isinstance(value, (int, float)) and
                value > 0 and value < 5):
            self._freq_scale_factor = value
        else:
            raise ValueError(f'Not valid frequency scale factor, got: {value}')

    @property
    def reaction_label(self):
        label = " + ".join(self.reactant_list)
        label += " <=> "
        label += " + ".join(self.product_list)
        return label
    
    @property
    def reactant_list(self):
        return [reactant[0] for reactant in self.reactants]
    
    @property
    def product_list(self):
        return [product[0] for product in self.products]

    def to_dict(self):
        return {'model_chemistry': self. model_chemistry.to_model_chem(),
                'freq_scale_factor': self.freq_scale_factor,
                'use_bond_corrections': self.use_bond_corrections,
                'freq_scale_factor': self.freq_scale_factor,
                'use_hindered_rotors': self.use_hindered_rotors,
                'reaction_label': self.reaction_label,
                'reactants': set(self.reactants),
                'products': set(self.products),
                'TS': self.TS,
                'reactant_list': self.reactant_list,
                'product_list': self.product_list,
                'tunneling': self.tunneling,
                'save_path': self.save_path,
                }


Enter the place where you store the species files

In [ ]:
database = '/Volumes/Extreme SSD/relax-rotor/Species'

### [OPTIONAL] A quick check if molecules are in the database (by searching the folder name)
Reture the coordinates if found

In [ ]:
reactants = [
    Molecule().from_smiles('O=CC(O)OO'),
]
species_base = os.listdir(database)
for mol in reactants:
    display(mol)
    smi = mol.to_smiles()
    if smi in species_base:
        print(f'{smi}: Yes')
        spc_path = os.path.join(database, smi)
        print(spc_path)
    else:
        print(f'{smi}: No')

## 1. Enter TS information

In [ ]:
# Where species data is stored
database = '/Volumes/Extreme SSD/relax-rotor/Arkane_Species_wo_bac'
# Where the current TS folder
cur_dir = '.'

# list all entries in the database
species_base = os.listdir(database)

reactants = [
    Molecule().from_smiles('CC(C)=C(C)C'),
#     Molecule().from_smiles('CCC(N)C=O'),
]

products = [
    Molecule().from_smiles('C=C(C)C(O[O])(C)C'),
#     Molecule().from_smiles('C=CC'),
]

# Switch to the current TS folder 
os.chdir(cur_dir)

### [OPTIONAL] rename all input.log file to output.out file

In [ ]:
# source_dir = '/Users/xiaorui/C3ddb server/Calcs/dow_16'
# for root, _, files in os.walk(source_dir):
#     for file in files:
#         if file == 'input.log':
#             dir_name = os.path.join(cur_dir, os.path.basename(root))
#             os.makedirs(dir_name, exist_ok=True)
#             shutil.copy(os.path.join(root, file), os.path.join(dir_name, 'output.out'))
        

In [ ]:
for root, _, files in os.walk(cur_dir):
    for file in files:
        if file == 'input.log':
            os.rename(os.path.join(root, file), os.path.join(root, 'output.out'))

## 2. Create a Arkane input file

In [ ]:
settings = {'reactants': [], 'products': [], 'TS': 'TS.py', 'tunneling': 'Eckart',
            'save_path': os.path.join(cur_dir, 'input.py')}

for label, mol in [('reactants', r) for r in reactants] + [('products', p) for p in products]:
    display(mol)
    smi = mol.to_smiles()
    if smi in species_base:
        print('Have a calculated entry')
        settings[label].append((smi, os.path.relpath(os.path.join(database, smi, 'species.py'))))
        # Optional print
#         print(f"'{os.path.relpath(os.path.join(database, smi, 'species.py'))}', structure=SMILES('{smi}'))")



Save the arkane input file for kinetics

In [ ]:
ArkaneKinetics({**settings}).save()

## 3. Create a TS.py file

Don't have to worry about the rotor information too much 

In [ ]:
spc = {'label': 'A',
       'directory': cur_dir,
       'ts': True}

ARKANE_SPEC = {
    'model_chemistry': 'cbs-qb3',
}

classify_jobs(spc)
find_latest_terminated_job(spc)
check_converge_and_geom_consist(spc)
generate_geom_info(spc)
find_rotors_from_xyz(spc)
filter_scans(spc, scan_filter='')
check_scan_quality(spc)
print(generate_summary(spc))
print(spc['smiles'])

In [ ]:
# Save path
save_dir = cur_dir
os.makedirs(save_dir, exist_ok=True)

# Change species directory to rel path according to the save path
change_to_relative_path(spc, save_dir)

# Save TS.py
arkane_species_path = os.path.join(save_dir, 'TS.py')
# arkane_thermo_input_path = os.path.join(save_dir, 'input.py')
settings = {'use_bond_corrections': False,
            'bond_dict': {},
            'save_path': arkane_species_path}
ArkaneSpecies({**spc, **ARKANE_SPEC, **settings}).save()

In [ ]:
import subprocess
cmd = f'python ~/Apps/RMG-Py/Arkane.py input.py'

subprocess.check_output(cmd,
                        cwd=cur_dir,
                        shell=True,
                        timeout=5000)

### [OPTIONAL] plot a TS rotor

In [ ]:
path = '/Users/xiaorui/Dropbox/RMG/Co-OPTIMA shared/relax-rotor/Species/CC(=O)OC(C)(C)C/scan_1_2_4_5/output.out'
energies, angles = parse_1d_scan_energies(path) 
plot_1d_rotor_scan(angles, energies)